## Spatially interpolate modeled LFM results to fill small gaps
Last updated: Kevin Varga, 11/27/2024

**Inputs:**
* Netcdf file of predicted LFM of all four fuels
* WRF land-sea mask

**Outputs:**
* Netcdf file of modeled and interpolated LFM of all four fuels

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.interpolate import griddata

In [2]:
# Set path
grid_path = '/home/sbarc/students/varga/nasa/ch1/data/lfm_model/'

In [3]:
# Open predicted LFM dataset
lfm_ds = xr.open_dataset(grid_path + 'predicted_ds.nc')
# Save fuel types
fuels = list(lfm_ds.data_vars)
# Save lat and lon points
lat = lfm_ds['latitude']
lon = lfm_ds['longitude']

In [4]:
# Use to identify time steps where the most nan values occur,
# verifying that interpolation is not filling in large areas

# Identify NaN values
nan_mask = lfm_ds['chamise'].isnull()
# Sum NaN values over each time step
nan_count = nan_mask.sum(dim=['latitude', 'longitude'])
# Save time step NaN counts to pandas series
nan_count_s = nan_count.to_series()
nan_over_15610 = nan_count_s[nan_count_s > 15610]

In [6]:
# Open wrf land-sea mask
land_mask = xr.open_dataarray('/home/sbarc/students/varga/nasa/ch1/data/wrf/vars/land_mask_crop.nc')

# Create mask for land
land_mask = (land_mask >= 1)
# Convert xarray mask to 1-D numpy mask
mask = land_mask.to_numpy().ravel()

In [45]:
%%time

for fuel_type in fuels:
    print(f'started {fuel_type}')
    for i in range(len(lfm_ds['time'])):
        # Ravel all points
        data = lfm_ds[fuel_type][i,:,:].to_numpy().ravel()
        # Mask to only land points
        points = data[mask]
        # Find the valid points
        valid = ~np.isnan(points)
        points_valid = points[valid]

        # Construct arrays of (x, y) points, masked to only include the valid points
        xx, yy = np.meshgrid(lon, lat)
        xx, yy = xx.ravel()[mask], yy.ravel()[mask]
        xxv = xx[valid]
        yyv = yy[valid]

        # Feed these points into the interpolator, and also provide the target grid
        interpolated = griddata(np.stack([xxv, yyv]).T, points_valid, (xx, yy), method="linear")

        # Plug the interpolated values back into the original data
        data[mask] = interpolated

        # Reassign values in dataset to interpolated values
        lfm_ds[fuel_type][i,:,:] = data.reshape(lfm_ds[fuel_type][i,:,:].shape)
        
# Save spatially interpolated LFM results
lfm_ds.to_netcdf(grid_path + 'interpolated_ds.nc')

started chamise
started chamise_old_growth
started sage_black
started ceanothus_bigpod
CPU times: user 9min 5s, sys: 3min 41s, total: 12min 46s
Wall time: 8min 50s
